In [1]:
from __future__ import division
import cympy
import pandas
import sys
import pickle

# Python API
sys.path.append("D:\\Users\\Jonathan\\Documents\\GitHub\\cymdist")
import function
import lookup

# Query database
from datetime import datetime
from pytz import timezone
from IPython.display import clear_output
pacific = timezone('US/Pacific')

In [2]:
# FMU initialize
# Shunt capacitor control
voltage_control = False

# Adding PV on the network
added_PV = False 

# Open a study
filename = 'C:\\Users\\emma\\Dropbox\\PG&E Models - DO NOT SHARE\\HL0004\\HL0004.sxst'
cympy.study.Open(filename)

# Set the grid model with the modification
if voltage_control:
    # Not possible yet!
    pass

if added_PV:
    my_pv = function.add_pv("New_PV", "BS2102-L_14036$P5080048-1_14036")

In [3]:
def run_simulation(datetime):
    """
    Args:
        datetime (datetime): datetime to querry database
    Retrun:
        high_voltage, low_voltage, overload, unbalanced (DataFrame)
    """
    # Retrieve microPMU data at t
    print('Queriyng information...')
    start = datetime.now()
    inputdt = pacific.localize(datetime)
    upmu_path = '/LBNL/grizzly_bus1/'
    udata = function.get_upmu_data(inputdt, upmu_path)
    end = datetime.now()
    print('Done in ' + str((end - start).total_seconds()) + ' seconds' )

    # Set the downstream load and voltage and launch load allocation
    print('Simulating...')
    start = datetime.now()
    function.load_allocation(udata)

    # Run a power flow
    lf = cympy.sim.LoadFlow()
    lf.Run()
    end = datetime.now()
    print('Done in ' + str((end - start).total_seconds()) + ' seconds' )

    # Get the results
    print('Saving results...')
    start = datetime.now()
    devices = function.list_devices()
    devices = function.get_voltage(devices)
    devices = function.get_overload(devices)
    devices = function.get_load(devices)
    devices = function.get_unbalanced_line(devices)
    devices = function.get_distance(devices)
    end = datetime.now()
    print('Done in ' + str((end - start).total_seconds()) + ' seconds' )
    
    return devices

In [4]:
# Input data
datetimes = [datetime(2016, 8, 25, i, 0, 0) for i in range(0, 24)]

# Result holder
result = []

start = datetime.now()
for datetime_value in datetimes:
#     clear_output()
    print('')
    print('###############')
    print(datetime_value)
    devices = run_simulation(datetime_value)
    result.append(devices)

end = datetime.now()
print('')
print('The whole simulation (' + str(len(datetimes)) + ' runs) took ' +
     str((end - start).total_seconds() / 60) + ' minutes')

# For each result drop the device column
for index, frame in enumerate(result):
    result[index] = frame.drop(['device'], axis=1)

# Saving results
filename = 'D://Users//Jonathan//Documents//GitHub//cymdist//result.pickle'
with open(filename, 'wb') as output:
    pickle.dump(result, output, protocol=2)


###############
2016-08-25 00:00:00
Queriyng information...
Done in 1.591035 seconds
Simulating...
Done in 0.891001 seconds
Saving results...
Done in 26.816041 seconds

###############
2016-08-25 01:00:00
Queriyng information...
Done in 1.089004 seconds
Simulating...
Done in 0.901001 seconds
Saving results...
Done in 26.735041 seconds

###############
2016-08-25 02:00:00
Queriyng information...
Done in 1.043003 seconds
Simulating...
Done in 0.895003 seconds
Saving results...
Done in 26.73404 seconds

###############
2016-08-25 03:00:00
Queriyng information...
Done in 0.890003 seconds
Simulating...
Done in 0.892002 seconds
Saving results...
Done in 26.785058 seconds

###############
2016-08-25 04:00:00
Queriyng information...
Done in 1.185005 seconds
Simulating...
Done in 0.902001 seconds
Saving results...
Done in 26.740042 seconds

###############
2016-08-25 05:00:00
Queriyng information...
Done in 1.133003 seconds
Simulating...
Done in 0.901001 seconds
Saving results...
Done in 26.77